# Creating a Siamese model: Ungraded Lecture Notebook

In this notebook you will learn how to create a siamese model in TensorFlow.

In [1]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model, Sequential
from tensorflow import math
import numpy

# Setting random seeds
numpy.random.seed(10)

2025-04-05 15:23:58.415677: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743888238.430455  151615 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743888238.434942  151615 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-05 15:23:58.447409: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Siamese Model

To create a `Siamese` model you will first need to create a LSTM model. For this you can stack layers using the`Sequential` model. To retrieve the output of both branches of the Siamese model, you can concatenate results using the `Concatenate` layer. You should be familiar with the following layers (notice each layer can be clicked to go to the docs):
   - [`Sequential`](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential) groups a linear stack of layers into a [`tf.keras.Model`](https://www.tensorflow.org/api_docs/python/tf/keras/Model)
   - [`Embedding`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding) Maps positive integers into vectors of fixed size. It will have shape (vocabulary length X dimension of output vectors). The dimension of output vectors (called `model_dimension`in the code) is the number of elements in the word embedding. 
   - [`LSTM`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/LSTM) The Long Short-Term Memory (LSTM) layer. The number of units should be specified and should match the number of elements in the word embedding. 
   - [`GlobalAveragePooling1D`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/GlobalAveragePooling1D) Computes global average pooling, which essentially takes the mean across a desired axis. GlobalAveragePooling1D uses one tensor axis to form groups of values and replaces each group with the mean value of that group. 
   - [`Lambda`](https://trax-ml.readthedocs.io/en/latest/trax.layers.html#trax.layers.base.Fn)  Layer with no weights that applies the function f, which should be specified using a lambda syntax. You will use this layer to apply normalization with the function
        - `tfmath.l2_normalize(x)`
        
- [`Concatenate`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Concatenate) Layer that concatenates a list of inputs. This layer will concatenate the normalized outputs of each LSTM into a single output for the model.
- [`Input`](https://www.tensorflow.org/api_docs/python/tf/keras/Input): it is used to instantiate a Keras tensor.. Remember to set correctly the dimension and type of the input, which are batches of questions. 


Putting everything together the Siamese model will look like this:

In [2]:
vocab_size = 500
model_dimension = 128

# Define the LSTM model
LSTM = Sequential()
LSTM.add(layers.Embedding(input_dim=vocab_size, output_dim=model_dimension))
LSTM.add(layers.LSTM(units=model_dimension, return_sequences=True))
LSTM.add(layers.GlobalAveragePooling1D())
LSTM.add(layers.Lambda(lambda x: math.l2_normalize(x)))

input1 = layers.Input((None,))
input2 = layers.Input((None,))

# Concatenate two LSTMs together
conc = layers.Concatenate(axis=1)((LSTM(input1), LSTM(input2)))
    
# Use the Parallel combinator to create a Siamese model out of the LSTM 
Siamese = Model(inputs=(input1, input2), outputs=conc)

# Print the summary of the model
Siamese.summary()

I0000 00:00:1743888240.800686  151615 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1590 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential          │ (None, 128)       │    195,584 │ input_layer[0][0… │
│ (Sequential)        │                   │            │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 256)       │          0 │ sequential[0][0], │
│ (Concatenate)       │                   │            │ sequential[1][0]  │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 195,584 (764.00 KB)

 Trainable params: 195,584 (764.00 KB)

 Non-trainable params: 0 (0.00 B)

Next is a helper function that prints information for every layer:

In [3]:
def show_layers(model, layer_prefix):
    print(f"Total layers: {len(model.layers)}\n")
    
    for i in range(len(model.layers)):
        print('========')
        print(f'{layer_prefix}_{i}: {model.layers[i]}\n')

print('Siamese model:\n')
show_layers(Siamese, 'Parallel.sublayers')

print('Detail of LSTM models:\n')
show_layers(LSTM, 'Serial.sublayers')

Siamese model:

Total layers: 4

Parallel.sublayers_0: <InputLayer name=input_layer, built=True>

Parallel.sublayers_1: <InputLayer name=input_layer_1, built=True>

Parallel.sublayers_2: <Sequential name=sequential, built=True>

Parallel.sublayers_3: <Concatenate name=concatenate, built=True>

Detail of LSTM models:

Total layers: 4

Serial.sublayers_0: <Embedding name=embedding, built=True>

Serial.sublayers_1: <LSTM name=lstm, built=True>

Serial.sublayers_2: <GlobalAveragePooling1D name=global_average_pooling1d, built=True>

Serial.sublayers_3: <Lambda name=lambda, built=True>



Try changing the parameters defined before the Siamese model and see how it changes!

You will actually train this model in this week's assignment. For now you should be more familiarized with creating Siamese models using TensorFlow. **Keep it up!**